In [137]:
import os
import ndjson
import re
from nltk.corpus import stopwords
import nltk
from string import digits
from collections import defaultdict
import gensim
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vitto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [148]:
# load from file-like objects
with open('realdonaldtrump_small.ndjson', encoding="utf8") as f:
    data = ndjson.load(f)

In [149]:
len(data)

12853

In [150]:
#Clean data
#Remove urls, stopwords, tweets less than 25 characters
def remove_urls(text):
    text = re.sub(r"(?:\@|http?\://)\S+", "", text)
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    return text

def filter_by_char_length(corpus, length=25):
    new_corpus=[]
    for doc in corpus:
        if len(doc) > length:
            new_corpus.append(doc)
    return new_corpus

def remove_stopwords(corpus):
    en_stops = set(stopwords.words('english'))
    en_stops.add("get")
    en_stops.add("back")
    en_stops.add("going")
    en_stops.add("many")
    en_stops.add("done")
    en_stops.add("want")
    en_stops.add("would")
    en_stops.add("never")
    en_stops.add("made")
    en_stops.add("ever")
    en_stops.add("way")
    en_stops.add("know")
    en_stops.add("even")
    en_stops.add("one")
    en_stops.add("two")
    en_stops.add("amp")
    en_stops.add("really")
    en_stops.add("much")

    new_corpus = []
    for doc in corpus:
        new_doc = []
        for word in doc:
            if word not in en_stops:
                new_doc.append(word)
        new_corpus.append(new_doc)
    return new_corpus

def tokenize_document(corpus):
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    new_corpus=[tokenizer.tokenize(doc.lower()) for doc in corpus]
    return new_corpus

def get_token_frequency(corpus):
    token_frequency = defaultdict(int)
    for doc in corpus:
        for token in doc:
            token_frequency[token] += 1
    return token_frequency

def remove_infrequent_words_digits_short_words(corpus, token_frequency, min_length=3):
    new_corpus=[]
    for doc in corpus:
        new_doc = []
        for word in doc:
            if token_frequency[word] > 1 and len(word.strip(digits)) == len(word) and len(word) >= min_length:
                new_doc.append(word)
        new_corpus.append(new_doc)
    return new_corpus

In [151]:
#Take only text to create corpus
corpus = [line["text"] for line in data]
corpus = [remove_urls(line) for line in corpus]
corpus = filter_by_char_length(corpus)
corpus = tokenize_document(corpus)
corpus = remove_stopwords(corpus)
token_frequency = get_token_frequency(corpus)
corpus = remove_infrequent_words_digits_short_words(corpus, token_frequency)

In [152]:
#Fit LDA model
id2word = corpora.Dictionary(corpus)
corpus = [id2word.doc2bow(doc) for doc in corpus]
print(len(id2word))
print(len(corpus))

6408
12238


In [153]:
n_topics = 3
lda_model = models.ldamodel.LdaModel(corpus=corpus,
                                     id2word=id2word,
                                     num_topics=n_topics, 
                                     random_state=100,
#                                      update_every=1,
                                     chunksize=100,
                                     passes=6,#n_topics,
                                     per_word_topics=True)

In [154]:
print(lda_model.print_topics())

[(0, '0.037*"president" + 0.030*"trump" + 0.014*"mueller" + 0.012*"media" + 0.009*"collusion" + 0.008*"democrats" + 0.007*"report" + 0.007*"russia" + 0.007*"obama" + 0.007*"far"'), (1, '0.053*"great" + 0.019*"thank" + 0.018*"today" + 0.018*"new" + 0.016*"united" + 0.015*"states" + 0.014*"america" + 0.012*"day" + 0.010*"state" + 0.010*"job"'), (2, '0.017*"border" + 0.015*"democrats" + 0.013*"news" + 0.010*"years" + 0.010*"mexico" + 0.009*"china" + 0.009*"fake" + 0.008*"time" + 0.008*"people" + 0.007*"deal"')]


In [155]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\vitto\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.087237 -0.278625       1        1  37.012245
1     -0.296690  0.067278       2        1  33.210495
0      0.209452  0.211347       3        1  29.777258, topic_info=     Category         Freq        Term        Total  loglift  logprob
2     Default  1901.000000       great  1901.000000  30.0000  30.0000
41    Default  1186.000000   president  1186.000000  29.0000  29.0000
19    Default   941.000000       trump   941.000000  28.0000  28.0000
10    Default   668.000000       thank   668.000000  27.0000  27.0000
200   Default   651.000000       today   651.000000  26.0000  26.0000
...       ...          ...         ...          ...      ...      ...
668    Topic3   149.471909    election   211.593323   0.8639  -5.3618
1435   Topic3   249.860077   democrats   850.333862  -0.0133  -4.8480
976    Topic3   133.679977    american   302.650818   0.3943  -5.4734
353    Topic3   126.933556        like   216.458740   0.6777  -5.5252
673    Topic3   122.137634  republican   216.905609   0.6371  -5.5637

[156 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1810      1  0.996547  administration
0         2  0.999480         america
976       1  0.323806        american
976       2  0.234594        american
976       3  0.442754        american
...     ...       ...             ...
8         1  0.100956            year
8         2  0.025239            year
8         3  0.873271            year
344       1  0.997262           years
768       2  0.997927            york

[169 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])